# 🥔 Étude Data Analyst : Le Grand Tournoi des Patates

Cette étude analyse les résultats de **500 000 combats** entre **1 000 patates** générées procéduralement.

## 📊 Sommaire
1. [Chargement des données](#1.-Chargement-des-données)
2. [Analyse du 'Meta' (Top Mutations/Armes)](#2.-Analyse-du-'Meta'-(Top-Mutations/Armes))
3. [Corrélation Statistiques vs Victoire](#3.-Corrélation-Statistiques-vs-Victoire)
4. [L'Anatomie du Champion (Top 1%)](#4.-L'Anatomie-du-Champion-(Top-1%))
5. [Le Triangle des Armes](#5.-Le-Triangle-des-Armes)
6. [Loi des Rendements Décroissants (Nombre de Traits)](#6.-Loi-des-Rendements-Décroissants-(Nombre-de-Traits))
7. [Analyse des Counters (Mutation vs Mutation)](#7.-Analyse-des-Counters-(Mutation-vs-Mutation))

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

plt.style.use('ggplot')
print("Bibliothèques chargées !")

## 1. Chargement des données
Nous chargeons les fichiers générés par la simulation.

In [ ]:
potatoes_df = pd.read_csv('potatoes.csv')
matches_df = pd.read_csv('matches.csv')

# Nettoyage des traits et armes (split des chaînes)
potatoes_df['trait_list'] = potatoes_df['traits'].fillna('').apply(lambda x: x.split('|') if x else [])
potatoes_df['weapon_list'] = potatoes_df['weapons'].fillna('').apply(lambda x: x.split('|') if x else [])
potatoes_df['num_traits'] = potatoes_df['trait_list'].apply(len)

print(f"Patates : {len(potatoes_df)}")
print(f"Matchs : {len(matches_df)}")
potatoes_df.head()

## 2. Analyse du 'Meta' (Top Mutations/Armes)
Calcul des performances globales.

In [ ]:
# Calcul du winrate par patate
wins = matches_df['winner_id'].value_counts().rename('wins')
p1_counts = matches_df['id_p1'].value_counts()
p2_counts = matches_df['id_p2'].value_counts()
total_games = p1_counts.add(p2_counts, fill_value=0).rename('total_games')

stats_df = potatoes_df.set_index('id').join(wins).join(total_games).fillna(0)
stats_df['winrate'] = (stats_df['wins'] / stats_df['total_games'].replace(0, 1)) * 100
stats_df['total_stats'] = stats_df[['force', 'vitesse', 'agilite', 'vitalite']].sum(axis=1)

print("Top 10 des patates les plus fortes :")
display(stats_df.sort_values('winrate', ascending=False).head(10))

## 3. Corrélation Statistiques vs Victoire
Quelle statistique est la plus déterminante ?

In [ ]:
plt.figure(figsize=(12, 6))
for stat in ['force', 'vitesse', 'agilite', 'vitalite']:
    sns.regplot(data=stats_df, x=stat, y='winrate', label=stat.capitalize(), scatter_kws={'alpha':0.1}, line_kws={'width':2})
plt.title("Influence des Statistiques sur le Winrate")
plt.legend()
plt.show()

correlations = stats_df[['force', 'vitesse', 'agilite', 'vitalite', 'winrate']].corr()['winrate'].sort_values(ascending=False)
print("Corrélations avec le winrate :")
print(correlations)

## 4. L'Anatomie du Champion (Top 1%)
Portrait-robot des 10 meilleures patates.

In [ ]:
top_1_percent = stats_df.sort_values('winrate', ascending=False).head(10)

print("--- Portrait-Robot du Champion ---")
print(f"Moyenne Force: {top_1_percent['force'].mean():.2f}")
print(f"Moyenne Vitesse: {top_1_percent['vitesse'].mean():.2f}")
print(f"Moyenne Mutations: {top_1_percent['num_traits'].mean():.2f}")

all_top_traits = [trait for sublist in top_1_percent['trait_list'] for trait in sublist]
print("\nTraits les plus fréquents chez les champions :")
print(pd.Series(all_top_traits).value_counts().head(5))

## 5. Le Triangle des Armes
Performance des armes les unes contre les autres.

In [ ]:
# Expansion des armes pour analyse
weapon_performance = {}
for weapon in stats_df['weapon_list'].explode().unique():
    if pd.isna(weapon): continue
    mask = stats_df['weapons'].str.contains(weapon, na=False)
    weapon_performance[weapon] = stats_df[mask]['winrate'].mean()

wp_series = pd.Series(weapon_performance).sort_values(ascending=False)
plt.figure(figsize=(10, 5))
wp_series.plot(kind='bar', color='skyblue')
plt.title("Winrate moyen par Arme")
plt.ylabel("Winrate (%)")
plt.show()

## 6. Loi des Rendements Décroissants (Nombre de Traits)
Est-ce que plus de mutations garantissent la victoire ?

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(data=stats_df, x='num_traits', y='winrate')
plt.title("Winrate en fonction du nombre de Mutations")
plt.xlabel("Nombre de Mutations")
plt.ylabel("Winrate (%)")
plt.show()

## 7. Analyse des Counters (Mutation vs Mutation)
Quelle mutation bat quoi ?

In [ ]:
# Cette section nécessite un pivot lourd sur les matchs.
# On va simplifier en regardant le winrate global des traits.
trait_performance = {}
for trait in stats_df['trait_list'].explode().unique():
    if pd.isna(trait): continue
    mask = stats_df['traits'].str.contains(trait, na=False)
    trait_performance[trait] = stats_df[mask]['winrate'].mean()

trait_series = pd.Series(trait_performance).sort_values(ascending=False)
print("Taux de victoire moyen par Mutation :")
display(trait_series.head(10))